# In this notebook i will make final fucntion which takes user query and gives top 10 most similar results.

## we have tried different embedding techniques in preprocessing_sc1.ipynb and we have found that Universal sentence encoder gives most similar results so that we will use universal sentence encoding model 

In [2]:
import pandas as pd
import joblib
import numpy as np
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import os
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
df = pd.read_csv('/content/drive/My Drive/self_case_study1/data/featurized_data.csv')
df.shape

(425167, 10)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test = train_test_split(df,test_size=0.5,random_state=42)

# reset index of train and test
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

print('Shape of X_train',X_train.shape)
print('Shape of X_test',X_test.shape)

Shape of X_train (212583, 10)
Shape of X_test (212584, 10)


In [ ]:
joblib.dump(X_train,'/content/drive/My Drive/self_case_study1/data/X_train.pkl')

['/content/drive/My Drive/self_case_study1/data/X_train.pkl']

In [3]:
# below mentioned functions are helper functions which is use for text preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\n", "", phrase)
    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def remove_stopwords(text):
    '''this function will remove stopwords from text '''
    final_text = ''
    for word in text.split():
        if word not in stopwords:
            final_text += word + ' '
    return final_text

def preprocess_title(text):
    # convert to lower case
    text = text.lower()
    # decontract
    text = decontracted(text)
    # remove all punctuations except a-z and c# and c++
    text = re.sub('[^a-zc#c++]+',' ',text)
    # remove stop words
    text = remove_stopwords(text)
    return text

In [18]:
tf.logging.set_verbosity(tf.logging.ERROR)
# load universal sentence encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def embed_text(sentences):
    ''' This function will convert sentence into vector using universal sentence encoder'''
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        # create vectors from sentences
        embeddings = session.run(embed(sentences))
        # return  embeddings
        return embeddings

In [13]:
# load X_train
X_train = joblib.load('/content/drive/My Drive/self_case_study1/data/X_train.pkl')
# load question embedding trained using universal sentence encoder
question_embeddings = joblib.load('/content/drive/My Drive/self_case_study1/data/universal_embed/universal_question_embedding.pkl')

In [21]:
def final_func1(question,top_n=10):
    ''' This function will find top similar result for given query using universal sentence encoder'''
    start = time.time()
    # preprocess question
    question = preprocess_title(question)
    # initialize  vector for user query
    query_vector = embed_text([question])
    similarities =  cosine_similarity((query_vector).reshape(1, -1), Y=question_embeddings, dense_output=True)
    # sort similarities 
    sort = np.argsort(similarities[0])
    # get top similarity indices  in descending order
    similarity_index = np.array(list(reversed(sort)))
    # find top n similarities
    top_similarity_index = similarity_index[:top_n]
    # print top similarity values
    print('Top cosine similarities are ======>',similarities[0][top_similarity_index])
    # get similar questions title
    similar_questions = X_train['title'][top_similarity_index]
    total_time = (time.time() - start)
    print('\t')
    print('Total time ===========> ',total_time)
    print('Top 10 similar questions Using Universal sentence encoder')
    print('='*100)
    return list(similar_questions)

In [22]:
query = 'How to make dictionary in python'
final_func1(query)

Top cosine similarities are ======> [0.76299614 0.71827483 0.7150197  0.677094   0.66860086 0.66528165
 0.6553011  0.6445547  0.6352596  0.6351328 ]
	
Total time ===========>  4.095801830291748
Top 10 similar questions Using Universal sentence encoder


['How to ignore \\ before " in simplejson?',
 'How to convert a list into a dictionary in python',
 'How to convert string to dictionary into python',
 'Python - Unnest List of Dictionaries',
 'How to use python dict in my write order',
 'How to make an OR statement in python?',
 'how to create a dict from two lists in python3',
 'write a dictionary of dictionaries (with unknown size) as a matrix',
 'Python converting a list into a dict with a value of 1 for each key',
 'Python - How to convert a list of dictionaries with tree items?']